## Assignment 1
### Name: Navaneeth Shaji
### Roll Number: 21CS30032

In [220]:
# import all the necessary libraries here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [221]:
df = pd.read_csv('../../dataset/decision-tree.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# normalising the data
st = StandardScaler() 
X_norm = st.fit_transform(X)

print(df.shape)


(768, 9)


In [222]:
# splitting the  dataset into training , validation and testing wihtout normalising the data
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=104,train_size=0.8,shuffle=True)

In [223]:
# function to find the entropy 
def entropy(y) : 
    x=np.count_nonzero(y==1)
    x=x/len(y)
    if x==1 or x==0 :
        return 0
    
    e= -x*np.log2(x) - (1-x)*np.log2(1-x)
    
    return e

In [224]:
# creating a node class for the tree
class NODE :
    def __init__(self):
        self.type="internal"
        self.entropy = None
        self.left = None
        self.right = None
        self.feature = None
        self.label = None

In [260]:
# finding the feature_split value for a feature

def feature_split(X,y,feature) :
    m = X.shape[0]
    x=X[:,feature]
    
    y_copy = y
    
    ind = np.argsort(x)
    x=x[ind]
    y_copy = y_copy[ind]
    
    feature_split_list = []
    
    for i in range(1,m) :
        if y_copy[i]!=y_copy[i-1] : 
            feature_split_list.append((x[i-1]+x[i])/2)
    
    entropy_list=[]
    
    
    
    for feature_split in feature_split_list :
        y_left=np.array([])
        y_right=np.array([])
        
        for i in range(m):
            if(x[i]<=feature_split) :
                y_left=np.append(y_left,y_copy[i])
            else :
                y_right=np.append(y_right,y_copy[i])
                
        if(len(y_left)==0 or len(y_right) == 0) :
                feature_split_list.remove(feature_split)
                continue 
        # the weighted sum of entropies of daughter nodes
        entropy_sum = len(y_left)*entropy(y_left)/len(y_copy) + len(y_right)*entropy(y_right)/len(y_copy)
        entropy_list.append(entropy_sum)
        
        
    if len(entropy_list) == 0 :
        return None ,None
    min_entropy_sum = np.min(entropy_list)
    t=np.argmin(entropy_list)
    feature_split = feature_split_list[t] 
    
    return min_entropy_sum , feature_split

In [255]:
# function to best feature 

def best_feature(X,y) :
    m=X.shape[0]
    n=X.shape[1]
    
    # stores the smallest entropy possible for each feature
    entropy_list=[]
    label_list = []
    
    for i in range(n) :
        entropy , label = feature_split(X,y,i)
        if entropy != None and label != None :
            entropy_list.append(entropy)
            label_list.append(label)
        
    
    min_entropy = np.min(entropy_list)
    feature = np.argmin(entropy_list)
    label = label_list[feature]
    
    return feature,min_entropy,label 

In [256]:
# function to build the decision tree 
def DecisionTree(X,y) :
    
    m=X.shape[0]
    n=X.shape[1]
    
    root = NODE()
    
    # base case 
    if m <10 or y.ptp() == 0 or n==0 :
        root.type = "leaf"
        values , counts = np.unique(y,return_counts=True)
        root.label=values[counts.argmax()]
        return root

    root.entropy = entropy(y)
    
    
    feature,entropy_sum,label = best_feature(X,y)
    
    information_gain = root.entropy - entropy_sum 
    root.feature = feature
    root.label = label 
    
    # splitting X and y based on the feature 
    X_left = np.empty((0,n))
    X_right = np.empty((0,n))
    y_left = np.array([])
    y_right = np.array([])
    
    for i in range(m) :
        if(X[i][root.feature] <= root.label) :
            X_left = np.r_[X_left,[X[i]]]
            y_left = np.append(y_left,y[i])
        else :
            X_right = np.r_[X_right,[X[i]]]
            y_right = np.append(y_right,y[i])
            
    if(len(y_left) == 0 or len(y_right)==0) :
        root.type = "leaf"
        values , counts = np.unique(y,return_counts=True)
        root.label=values[counts.argmax()]
        return root
        
    # removing column feature from X_left and X_right 
    X_left = np.delete(X_left,feature,1)
    X_right = np.delete(X_right,feature,1)
    
    root.left = DecisionTree(X_left,y_left)
    root.right = DecisionTree(X_right,y_right)
    
    return root
    

In [261]:
root=DecisionTree(X_train,y_train)

In [262]:


def printtree(root) : 
    if(root.type == "leaf") :
        print(root.type,root.entropy,root.label,root.feature)
        return

    printtree(root.left)
    print(root.type,root.entropy,root.label,root.feature)
    printtree(root.right)

printtree(root)

internal 0.9412682104495973 127.5 1
internal 0.726053856943508 28.0 6
internal 0.42061170899770106 31.0 4
internal 0.16866093149667025 7.0 0
internal 0.12311463565031058 47.0 2
leaf None 0.0 None
internal 0.23519338181924143 62.0 0
internal 0.39124356362925566 31.0 0
internal 0.2580186686648155 0.6575 0
leaf None 0.0 None
leaf None 0.0 None
leaf None 0.0 None
leaf None 0.0 None
leaf None 1.0 None
internal 0.6457523329916602 37.0 1
leaf None 1.0 None
internal 0.5999108763872214 0.0 2
internal 0.863120568566631 5.5 0
internal 0.9182958340544896 33.5 0
internal 0.8453509366224364 0.1285 0
leaf None 1.0 None
leaf None 0.0 None
leaf None 1.0 None
leaf None 0.0 None
internal 0.40907313904382653 0.5115000000000001 2
internal 0.1623261801753929 36.0 1
internal 0.222284830685688 2.0 0
leaf None 0.0 None
leaf None 0.0 None
leaf None 0.0 None
internal 0.7424875695421236 3.5 0
internal 0.5225593745369408 29.5 0
leaf None 0.0 None
leaf None 0.0 None
leaf None 1.0 None
internal 0.9275265884316759 26

In [264]:
# defining the prediction function for a test case
def predicted(root,x):
    if root.type == "leaf" :
        return root.label 
    
    if x[root.feature] <= root.label :
        pred = predicted(root.left,x)
    else :
        pred = predicted(root.right,x)
    
    return pred

In [265]:
# ontaining the predicted values over the test set 

y_pred = np.array([])

m = X_test.shape[0] 

for i in range(m) :
    test = predicted(root,X[i])
    y_pred = np.append(y_pred,test)

print(y_pred,y_test)

[0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 1. 0. 1. 1.] [0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0
 1 1 0 1 0 0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0
 0 0 1 1 0 1]


In [ ]:
# making the confusion matrix

